In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
raw_data = pd.read_csv('../DataSets/winequality-red.csv')

label =  np.unique(raw_data['quality'])
label

array([3, 4, 5, 6, 7, 8], dtype=int64)

In [3]:
train_data = raw_data.iloc[ : int(raw_data.shape[0] * .75) , :]
test_data = raw_data.iloc[int(raw_data.shape[0] * .75) : , :]
train_data_labels = train_data['quality']
test_data_labels = test_data['quality']
train_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
del train_data['quality'] , test_data['quality']

In [5]:
# Standardize data
train_data_normalize = ((train_data - np.mean(train_data)) / np.std(train_data))
test_data_normalize = ((test_data - np.mean(test_data)) / np.std(test_data))
train_data_normalize.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.687948,1.009682,-1.491607,-0.525724,-0.274715,-0.415701,-0.379669,0.394939,1.354414,-0.601306,-0.900712
1,-0.463362,2.014523,-1.491607,0.028111,0.180864,0.956022,0.592778,-0.137748,-0.631550,0.081104,-0.534222
2,-0.463362,1.344629,-1.288220,-0.209247,0.056615,-0.023780,0.209693,-0.031210,-0.247170,-0.089499,-0.534222
3,1.445622,-1.334945,1.355818,-0.525724,-0.295423,0.172180,0.386501,0.501476,-0.887804,-0.487571,-0.534222
4,-0.687948,1.009682,-1.491607,-0.525724,-0.274715,-0.415701,-0.379669,0.394939,1.354414,-0.601306,-0.900712


In [7]:
train_data_normalize = np.array(train_data_normalize)
train_data_labels = np.array(train_data_labels)

test_data_normalize = np.array(test_data_normalize)
test_data_labels = np.array(test_data_labels)

In [ ]:
class NeuralNetwork():
    
    def __init__(self,lmda = 0):
        
        self.input_layer_n   = 11
        
        self.hidden_layer1_n = 11
        self.hidden_layer2_n = 11
        self.hidden_layer3_n = 11
        self.hidden_layer4_n = 11
        self.hidden_layer5_n = 11
        
        self.output_layer = 6
        self.lmda = lmda
        self.label_length = 6
        np.random.seed(40)

        
        self.theta_layer1 = np.random.randn(self.input_layer_n   , self.hidden_layer1_n)
        self.theta_layer2 = np.random.randn(self.hidden_layer1_n , self.hidden_layer2_n)
        self.theta_layer3 = np.random.randn(self.hidden_layer2_n , self.hidden_layer3_n)
        self.theta_layer4 = np.random.randn(self.hidden_layer3_n , self.hidden_layer4_n)
        self.theta_layer5 = np.random.randn(self.hidden_layer4_n , self.hidden_layer5_n)
        self.theta_layer6 = np.random.randn(self.hidden_layer5_n , self.output_layer)
        
        self.bias1 = np.random.randn(self.hidden_layer1_n)
        self.bias2 = np.random.randn(self.hidden_layer2_n)
        self.bias3 = np.random.randn(self.hidden_layer3_n)
        self.bias4 = np.random.randn(self.hidden_layer4_n)
        self.bias5 = np.random.randn(self.hidden_layer5_n)
        self.bias6 = np.random.randn(self.output_layer)
       
        
    def one_hot_label(self , data , label_length):
        one_hot_labels = np.zeros((data.shape[0] , label_length))
        for i in range(data.shape[0]):  
            one_hot_labels[i, data[i]-3 ] = 1  # our labels is 3,4,5,6,7,8 is we subtract by 3 it is 0,1,2,3,4,5
        return one_hot_labels


    def softmax(self , activation , theta , bias):
        exp = np.exp( np.dot( activation , theta )  + bias )
        denominator = np.sum( exp , axis = 1)
        s = list(map(lambda i : exp[:,i] / denominator , np.arange(self.label_length)))
        return np.array(s).T
        
    def relu(self,activation,theta, bias):
        q = np.dot(activation , theta) + bias
        return np.clip(q , a_min=0 , a_max = q)
    
    def forwardPropagation(self,X):
        self.activation_layer_0 = X
        self.activation_layer_1 = self.relu(self.activation_layer_0 , self.theta_layer1 , self.bias1)
        self.activation_layer_2 = self.relu(self.activation_layer_1 , self.theta_layer2 , self.bias2)
        self.activation_layer_3 = self.relu(self.activation_layer_2 , self.theta_layer3 , self.bias3)
        self.activation_layer_4 = self.relu(self.activation_layer_3 , self.theta_layer4 , self.bias4)
        self.activation_layer_5 = self.relu(self.activation_layer_4 , self.theta_layer5 , self.bias5)
        self.activation_layer_6 = self.softmax(self.activation_layer_5 , self.theta_layer6 , self.bias6)
        
        return self.activation_layer_6
        
    def costFunction(self,X,y):
        ''' Provide labels as "Y" '''
        
        self.y = self.one_hot_label( y , self.label_length)
        self.y_pred = self.forwardPropagation(X)
        #np.nan_to_num(self.y_pred , copy = False)
        self.crossEntropy = self.y * np.log(self.y_pred)
        np.nan_to_num(self.crossEntropy, copy=False)
        return -np.sum(self.crossEntropy)
     
    
    def costFunctionDerivative(self,X,y):
        
        self.y_cap = self.forwardPropagation(X)
        self.target_y = self.one_hot_label(y , self.label_length)
        
        dL_dA6 = (self.y_cap - self.target_y)
        dL_dA5 = np.dot( dL_dA6 , self.theta_layer6.T )
        dL_dA4 = np.dot( dL_dA5 , self.theta_layer5.T )
        dL_dA3 = np.dot( dL_dA4 , self.theta_layer4.T )
        dL_dA2 = np.dot( dL_dA3 , self.theta_layer3.T )
        dL_dA1 = np.dot( dL_dA2 , self.theta_layer2.T )
        
        
        dL_dtheta_layer_6 =  1/X.shape[0] * (np.dot(self.activation_layer_5.T , dL_dA6)) 
        dL_dtheta_layer_5 =  1/X.shape[0] * (np.dot(self.activation_layer_4.T , dL_dA5))
        dL_dtheta_layer_4 =  1/X.shape[0] * (np.dot(self.activation_layer_3.T , dL_dA4))
        dL_dtheta_layer_3 =  1/X.shape[0] * (np.dot(self.activation_layer_2.T , dL_dA3))
        dL_dtheta_layer_2 =  1/X.shape[0] * (np.dot(self.activation_layer_1.T , dL_dA2))
        dL_dtheta_layer_1 =  1/X.shape[0] * (np.dot(self.activation_layer_0.T , dL_dA1))       
        
        dL_dbias6 =  1/X.shape[0] * dL_dA6 
        dL_dbias5 =  1/X.shape[0] * dL_dA5
        dL_dbias4 =  1/X.shape[0] * dL_dA4
        dL_dbias3 =  1/X.shape[0] * dL_dA3
        dL_dbias2 =  1/X.shape[0] * dL_dA2
        dL_dbias1 =  1/X.shape[0] * dL_dA1
        
         
        return dL_dtheta_layer_1, dL_dtheta_layer_2, dL_dtheta_layer_3 , dL_dtheta_layer_4 , dL_dtheta_layer_5, dL_dtheta_layer_6 , dL_dbias1 , dL_dbias2 ,dL_dbias3 , dL_dbias4 , dL_dbias5 ,  dL_dbias6
      

In [ ]:
def gradientDecent(NN , X , y , batch ):
    alpha = 0.01
    epsilon = 0.0001
    i = 1
    itera = []
    csfun = []
    while i < 1000:
        index = np.random.randint(0 , X.shape[0] , size=(batch,))
        
        initial_cost = NN.costFunction(X[index] , y[index] )
       
        theta1 , theta2 ,theta3 , theta4, theta5 ,  theta6 , bias1 , bias2 ,  bias3 , bias4 , bias5 , bias6 = NN.costFunctionDerivative(X[index], y[index] )
        
        NN.theta_layer1 -=  alpha * theta1
        NN.theta_layer2 -=  alpha * theta2
        NN.theta_layer3 -=  alpha * theta3
        NN.theta_layer4 -=  alpha * theta4
        NN.theta_layer5 -=  alpha * theta5
        NN.theta_layer6 -=  alpha * theta6
        
        NN.bias1 -= alpha * bias1.sum(axis = 0)
        NN.bias2 -= alpha * bias2.sum(axis = 0)
        NN.bias3 -= alpha * bias3.sum(axis = 0)
        NN.bias4 -= alpha * bias4.sum(axis = 0)
        NN.bias5 -= alpha * bias5.sum(axis = 0)
        NN.bias6 -= alpha * bias6.sum(axis = 0)
        
        
        final_cost = NN.costFunction(X[index] , y[index])
        
        #if abs(final_cost - initial_cost) < epsilon:
         #   break
        print('Iteration {} Initial cost {} final cost {}'.format(i , initial_cost , final_cost))    
        
        itera.append(i)
        csfun.append(final_cost)
        i+=1
    return itera , csfun

In [ ]:
NN = NeuralNetwork()
itera , csfun = gradientDecent(NN , train_data_normalize , train_data_labels , 32)

In [ ]:
plt.plot(itera , csfun)

In [ ]:
qq = (np.argmax(NN.forwardPropagation(train_data_normalize) , axis=1) +3)
co=0
for i in range(len(train_data_labels)):
    if qq[i] == train_data_labels[i]:
        co+=1
print('accuracy :')
print((co/len(train_data_labels))*100)

In [ ]:
qq = (np.argmax(NN.forwardPropagation(test_data_normalize) , axis=1) +3)
co=0
for i in range(len(test_data_labels)):
    if qq[i] == test_data_labels[i]:
        co+=1
print('accuracy : ')
print((co/len(test_data_labels))*100)

In [ ]:
z  = (np.argmax(NN.forwardPropagation(train_data_normalize) , axis=1) + 3) 
sns.countplot(z)

In [ ]:

def softmax(activation , theta , bias):
        exp = np.exp( np.dot( activation , theta )  + bias )
        denominator = np.sum( exp , axis = 1)
        s = list(map(lambda i : exp[:,i] / denominator , np.arange(self.label_length)))
        return np.array(s).T
NN = NeuralNetwork()
pd.DataFrame(NN.forwardPropagation(train_data_normalize))

index = np.random.randint(0 , train_data_normalize.shape[0] , size=(18,))
    
NN.costFunction(train_data_normalize[index] , train_data_labels[index])



In [ ]:
np.random.randn(5,5)